# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'Heart_Disease_Kaggle_AutoML'

experiment=Experiment(ws, experiment_name)
print("Experiment created")

run = experiment.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-136928
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-136928
Experiment created


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Creating a compute cluster
# vm_size = "Standard_D2_V2"
# max_nodes = 4.

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Exploring the Dataset

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

url_path = "https://raw.githubusercontent.com/bharati-21/AZMLND-Capstone-Project/master/files/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

df = pd.read_csv(url_path)

In [5]:
ds_dataframe = ds.to_pandas_dataframe()
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x.head()

Shape of dataset before split: (302, 21)


,age,resting_BP,cholesterol,max_heart_rate,st_depression,sex_1,chest_pain_type_1,chest_pain_type_2,chest_pain_type_3,fasting_blood_sugar_1,rest_ECG_1,rest_ECG_2,exercise_induced_angina_1,st_slope_1,st_slope_2,num_major_vessels_1.0,num_major_vessels_2.0,num_major_vessels_3.0,thalassemia_2.0,thalassemia_3.0
0,63,145,233,150,2.3,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
1,37,130,250,187,3.5,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
2,41,130,204,172,1.4,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
3,56,120,236,178,0.8,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0
4,57,120,354,163,0.6,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0


In [ ]:
# x['target'] = y
# x

In [12]:
import os
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory 'Data' created...")
except OSError:
    print("Directory 'Data' cannot be created...")

Dicrectory 'Data' created...


In [13]:
# split data into test and train sets

from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)
print(test_x.shape, train_x.shape, test_y.shape, train_y.shape)

train_df = train_x.copy(deep=True)
train_df['target'] = train_y
print(train_df.shape)

test_df = test_x.copy(deep=True)
test_df['target'] = test_y
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv", index = False)
test_df.to_csv("./Data/test_data.csv", index = False)

(61, 20) (241, 20) (61,) (241,)
(241, 21)
(61, 21)


In [14]:
deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="Data", target_path="Data", overwrite=True)

Uploading an estimated of 2 files
Uploading Data/test_data.csv
Uploaded Data/test_data.csv, 1 files out of an estimated total of 2
Uploading Data/train_data.csv
Uploaded Data/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_82eea1980fca4576912f61663c691301

In [15]:
from azureml.core import Dataset

train_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/train_data.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/test_data.csv'))])


In [16]:
train_data.to_pandas_dataframe()

,age,resting_BP,cholesterol,max_heart_rate,st_depression,sex_1,chest_pain_type_1,chest_pain_type_2,chest_pain_type_3,fasting_blood_sugar_1,...,rest_ECG_2,exercise_induced_angina_1,st_slope_1,st_slope_2,num_major_vessels_1.0,num_major_vessels_2.0,num_major_vessels_3.0,thalassemia_2.0,thalassemia_3.0,target
0,64,110,211,144,1.8,1,0,0,1,0,...,0,1,1,0,0,0,0,1,0,1
1,66,160,228,138,2.3,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,55,140,217,111,5.6,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,44,130,219,188,0.0,1,1,0,0,0,...,0,0,0,1,0,0,0,1,0,1
4,41,110,172,158,0.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,48,130,245,180,0.2,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,1
237,59,164,176,90,1.0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
238,62,150,244,154,1.4,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
239,65,150,225,114,1.0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [17]:
train_data = train_data.drop_columns('Column1')
train_data, train_data.to_pandas_dataframe().shape

({
   "source": [
     "('workspaceblobstore', 'Data/train_data.csv')"
   ],
   "definition": [
     "GetDatastoreFiles",
     "ParseDelimited",
     "DropColumns",
     "SetColumnTypes",
     "DropColumns"
   ]
 },
 (241, 21))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [18]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_config = AutoMLConfig (
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = train_data,
    label_column_name = "target",
    n_cross_validations = 5,
    enable_onnx_compatible_models = True,
    compute_target = cpu_cluster)

print("AutoML config created.")

AutoML config created.


In [19]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_161ff06f-d36e-44f5-b85a-6136c55e260f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values we

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_161ff06f-d36e-44f5-b85a-6136c55e260f',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-02T10:09:00.557528Z',
 'endTimeUtc': '2021-02-02T10:53:10.450047Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Heart_Disease_Kaggle_AutoML","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-136928","workspace_name":"quick-starts-ws-136928","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_cl

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [21]:
best_run, fitted_model = remote_run.get_output()
print(best_run)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: Heart_Disease_Kaggle_AutoML,
Id: AutoML_161ff06f-d36e-44f5-b85a-6136c55e260f_26,
Type: azureml.scriptrun,
Status: Completed)


In [22]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.33789473684210525,
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=100,
           

In [23]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":2,"CurrentNodeCount":2}',
 'ensembled_iterations': '[17, 0, 23, 6, 1, 21, 25]',
 'ensembled_algorithms': "['ExtremeRandomTrees', 'LightGBM', 'RandomForest', 'GradientBoosting', 'XGBoostClassifier', 'XGBoostClassifier', 'ExtremeRandomTrees']",
 'ensemble_weights': '[0.07142857142857142, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.21428571428571427, 0.14285714285714285, 0.21428571428571427]',
 'best_individual_pipeline_score': '0.8172619047619047',
 'best_individual_iteration': '17',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_161ff06f-d36e-44f5-b85a-6136c55e260f_ModelExplain',
 'model_explanation': 'True'}

In [25]:
import joblib
from azureml.core.model import Model

#TODO: Save the best model

description = "Best Model from AutoML run of the Kaggle Heart Disease Dataset"
automl_model = remote_run.register_model(model_name='kaggle_heart_autoML_best_model.pkl', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service